In [73]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
# fig, ax = plt.subplots()
# ax.plot(df, label='a')
# ax.set_ylabel('capacity(MW)')
# ax.set_title('result')
# ax.legend()
# plt.show()

In [62]:
def readCSV():
    df = pd.read_csv('20190101-20201031.csv', usecols=['備轉容量(MW)','淨尖峰供電能力(MW)', '尖峰負載(MW)', '備轉容量率(%)'])
    df = pd.DataFrame(df.values, columns=['備轉容量(MW)','淨尖峰供電能力(MW)', '尖峰負載(MW)', '備轉容量率(%)'])
    date_format = pd.read_csv('20190101-20201031.csv', usecols=['日期'])
    date_format = pd.to_datetime(date_format['日期'], yearfirst=True, format='%Y%m%d')
    df['month'] = date_format.dt.month
    df['day'] = date_format.dt.day
    df['dayOfWeek'] = date_format.dt.dayofweek
    return df
train_data = readCSV()
train_data.shape

(670, 7)

In [63]:
def normalize(data):
    data_norm = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return data_norm
data_norm = normalize(train_data)
data_norm

,備轉容量(MW),淨尖峰供電能力(MW),尖峰負載(MW),備轉容量率(%),month,day,dayOfWeek
0,-0.374294,-0.347947,-0.133519,-0.064174,-0.460516,-0.491244,-0.333333
1,-0.190639,-0.111495,-0.221385,-0.254193,-0.460516,-0.457910,-0.166667
2,-0.175614,-0.094685,-0.222453,-0.258992,-0.460516,-0.424577,0.000000
3,-0.177746,-0.092583,-0.233136,-0.272907,-0.460516,-0.391244,0.166667
4,-0.296731,-0.227288,-0.220850,-0.227802,-0.460516,-0.357910,0.333333
...,...,...,...,...,...,...,...
665,0.064386,0.083207,-0.020682,-0.062735,0.357666,0.375423,-0.333333
666,0.082559,0.099244,-0.011602,-0.056976,0.357666,0.408756,-0.166667
667,0.059665,0.061199,0.020046,-0.008992,0.357666,0.442090,0.000000
668,0.034335,0.005514,0.087881,0.090336,0.357666,0.475423,0.166667


In [64]:
def setTrainTestData(data, ref_day, predict_day, ratio):
    x, y = [], []
    for i in range(len(data) - ref_day - predict_day):
        x.append(np.array(data.iloc[i:i + ref_day]))
        y.append(np.array(data.iloc[i + ref_day:i + ref_day + predict_day]['備轉容量(MW)']))
    x,y = np.array(x), np.array(y)
    x_train = x[:int(x.shape[0] * ratio)]
    x_test = x[int(x.shape[0] * ratio):]
    y_train = y[:int(y.shape[0] * ratio)]
    y_test = y[int(y.shape[0] * ratio):]
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = setTrainTestData(data_norm, 7, 7, 0.9)
y_test.shape


(66, 7)

In [65]:

def shuffle(x, y):
    np.random.seed(int(time.time()))
    randomList = np.arange(x.shape[0])
    np.random.shuffle(randomList)
    return x[randomList], y[randomList]
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)
x_train

array([[[-0.07815213, -0.05525764, -0.06127668, ..., -0.27869742,
         -0.09124378,  0.16666667],
        [-0.21190848, -0.24122313,  0.03593645, ..., -0.27869742,
         -0.05791045,  0.33333333],
        [-0.2777968 , -0.29458522, -0.00853054, ..., -0.27869742,
         -0.02457711,  0.5       ],
        ...,
        [-0.04602015, -0.07804021,  0.0782669 , ..., -0.27869742,
          0.04208955, -0.33333333],
        [-0.02698462, -0.05785662,  0.07960224, ..., -0.27869742,
          0.07542289, -0.16666667],
        [-0.06744147, -0.042871  , -0.06301263, ..., -0.27869742,
          0.10875622,  0.        ]],

       [[ 0.31428442,  0.29416727,  0.12727407, ...,  0.08493894,
          0.10875622,  0.16666667],
        [ 0.15397985,  0.02508963,  0.35535104, ...,  0.08493894,
          0.14208955,  0.33333333],
        [ 0.06047731, -0.03098203,  0.24478445, ...,  0.08493894,
          0.17542289,  0.5       ],
        ...,
        [ 0.30291386,  0.26762447,  0.16145891, ...,  

In [71]:

def buildManyToManyModel(length, dim):
  model = Sequential()
  model.add(LSTM(32, input_shape=(length, dim), return_sequences=True))
  # output shape: (7, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
  model.summary()
  return model

model = buildManyToManyModel(7, 7)
filepath = "weights-{epoch:02d}-{accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')

NotImplementedError: Cannot convert a symbolic Tensor (lstm_5/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported